In [6]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from pandas import DataFrame
from sklearn.neighbors import KNeighborsClassifier
import scipy

In [7]:
def get_im_cv2(path):
    img = cv2.imread(path)
    height, width, channels = img.shape
    img = cv2.resize(img, (64, 64))
    return img

In [8]:
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['Automobile',
               'Comics_and_Cartoons',
               'Famous_Personalities',
               'Festivals_and_Occasions',
               'Graffiti_and_Illustrations',
               'Movies_and_TV_shows',
               'Music',
               'Nature',
               'No_Theme',
               'Patterns_and_Ethnic',
               'Signs_and_Symbols',
               'Spiritual',
               'Sports',
               'Superheroes',
               'Typography',
               'Vintage'
              ]
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Train', fld, '*.jpg')
        files = glob.glob(path)
        for i in range(len(files)):
            flbase = os.path.basename(files[i])
            img = get_im_cv2(files[i])
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)   #what does this do?
    print len(X_train)
    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

In [9]:
def load_test():
    path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

In [10]:
Xtrall, Ytrall, Itrall = load_train()
X_test, X_test_id = load_test()

Read train images
Load folder Automobile (Index: 0)
Load folder Comics_and_Cartoons (Index: 1)
Load folder Famous_Personalities (Index: 2)
Load folder Festivals_and_Occasions (Index: 3)
Load folder Graffiti_and_Illustrations (Index: 4)
Load folder Movies_and_TV_shows (Index: 5)
Load folder Music (Index: 6)
Load folder Nature (Index: 7)
Load folder No_Theme (Index: 8)
Load folder Patterns_and_Ethnic (Index: 9)
Load folder Signs_and_Symbols (Index: 10)
Load folder Spiritual (Index: 11)
Load folder Sports (Index: 12)
Load folder Superheroes (Index: 13)
Load folder Typography (Index: 14)
Load folder Vintage (Index: 15)
4592
Read train data time: 87.63 seconds


In [11]:
# flatten out all images to be one-dimensional

Xtrall = np.array(Xtrall, dtype=np.uint8)
Ytrall = np.array(Ytrall, dtype=np.uint8)
X_test =  np.array(X_test, dtype=np.uint8)

Xtrall = Xtrall.astype('float32')
# Ytrall = Ytrall.astype('float32')
X_test = X_test.astype('float32')
# Xtrall = Xtrall / 255

msk = np.random.rand(len(Xtrall)) < 0.8
Xval = Xtrall[~msk]
Yval = Ytrall[~msk]
Ytr = Ytrall
Xtr = Xtrall

# Preprocessing: reshape the image data into rows
Xtr = np.reshape(Xtr, (Xtr.shape[0], -1)) 
Xval = np.reshape(Xval, (Xval.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
Ytr = np.reshape(Ytr, (Ytr.shape[0], -1))

# Normalize the data: subtract the mean image
Xtr -= np.mean(Xtr, axis = 0)
Xval -= np.mean(Xval, axis = 0)
X_test -= np.mean(X_test, axis = 0)
Xtr /= np.std(Xtr, axis=0)
Xval /= np.std(Xval, axis=0)
X_test /= np.std(X_test, axis=0)
 
# Add bias dimension and transform into columns
# Xtr = np.hstack([Xtr, np.ones((Xtr.shape[0], 1))])
# Xval = np.hstack([Xval, np.ones((Xval.shape[0], 1))])
# X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])


In [12]:
print 'Train data shape: ', Xtr.shape
print 'Train labels shape: ', Ytr.shape
print 'Validation data shape: ', Xval.shape
print 'Validation labels shape: ', Yval.shape
print 'Test data shape: ', X_test.shape

Train data shape:  (4592L, 12288L)
Train labels shape:  (4592L, 1L)
Validation data shape:  (892L, 12288L)
Validation labels shape:  (892L,)
Test data shape:  (1200L, 12288L)


In [13]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(Xtr, Ytr)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [18]:
X_test_predict = clf.predict(X_test)


In [19]:
DataFrame(X_test_id,X_test_predict).to_csv("nn_submit_svmsgd.csv")

In [24]:

from sklearn import svm
clf = svm.SVC(gamma=0.001)
clf = clf.fit(Xtr, Ytr)

In [25]:
X_test_predict = clf.predict(X_test)

In [26]:
DataFrame(X_test_id,X_test_predict).to_csv("nn_submit_svmsvcgamma0.001.csv")